In [1]:
from cloudfiles import CloudFiles
import torch
from cloudvolume import CloudVolume
import zetta_utils as zu
zu.load_all_modules()

/home/nkemnitz/zetta/zetta_utils/venv-3.11/lib/python3.11/site-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [3]:
ANCHOR_Z_MAP = {
    0: 24,
    1: 124,
    2: 224,
    3: 324,
    4: 424,
    5: 524,
    6: 624,
    7: 725,
    8: 825,
    9: 924,
    10: 1024,
    11: 1124,
    12: 1224,
    13: 1325,
    14: 1424,
    15: 1524,
    16: 1624,
    17: 1739,
    18: 1836,
    19: 1936,
    20: 2036,
    21: 2136,
    22: 2236,
    23: 2740,
    24: 2840,
    25: 2940,
    26: 3040,
    27: 3140,
    28: 3240,
    29: 3340,
    30: 3440,
    31: 3540,
    32: 3640,
    33: 3740,
    34: 3840,
    35: 3940,
    36: 3993,
}

In [5]:
ANCHOR_Z_MAP

{0: 24,
 1: 124,
 2: 224,
 3: 324,
 4: 424,
 5: 524,
 6: 624,
 7: 725,
 8: 825,
 9: 924,
 10: 1024,
 11: 1124,
 12: 1224,
 13: 1325,
 14: 1424,
 15: 1524,
 16: 1624,
 17: 1739,
 18: 1836,
 19: 1936,
 20: 2036,
 21: 2136,
 22: 2236,
 23: 2740,
 24: 2840,
 25: 2940,
 26: 3040,
 27: 3140,
 28: 3240,
 29: 3340,
 30: 3440,
 31: 3540,
 32: 3640,
 33: 3740,
 34: 3840,
 35: 3940,
 36: 3993}

In [7]:

field_inv_cv = CloudVolume("gs://zetta-research-nico/hippocampus/rigid_w_scale/low_res_enc_c4_rigid", mip=[6144,6144,45], fill_missing=True, progress=False)
field_inv_small_cv = CloudVolume("gs://zetta-research-nico/hippocampus/rigid_w_scale/low_res_enc_c4_rigid_100th", mip=[6144,6144,45], delete_black_uploads=True, cdn_cache=False)
field_inv_small_cv.info['scales'][0]['size'] = [256, 256, 37]
field_inv_small_cv.info['scales'][0]['voxel_offset'] = [0, 0, 0]
field_inv_small_cv.commit_info()

for dst_z, src_z in ANCHOR_Z_MAP.items():
    field_inv_small_cv[:,:,dst_z,:] = field_inv_cv[:,:,src_z,:]



Uploading: 100%|██████████| 1/1 [00:00<00:00,  3.10it/s]


In [ ]:
from zetta_utils.alignment.aced_relaxation import perform_aced_relaxation
perform_aced_relaxation(
    match_offsets=(
        torch.ones((1,1,128,128,35))
    ),
    pfields={
        "-1": field_inv_cv
    },
    num_iter=4000,
    lr=1e-4,
    rigidity_weight=1e-3,
    fix='first',
    max_dist=1,
)

In [10]:
from copy import deepcopy
field_aced_condensed_cv = CloudVolume("gs://zetta-research-nico/hippocampus/aced_coarse_100th_w_scale/afield_try_12288nm_iter20000_rig300_lr0.0001_final", mip=[12288,12288,45], fill_missing=True, progress=False)
# field_aced_cv_info = deepcopy(CloudVolume("gs://zetta-research-nico/hippocampus/aced_coarse/coarse_anchors/field",  delete_black_uploads=True, cdn_cache=False).info)
# field_aced_cv = CloudVolume("gs://zetta-research-nico/hippocampus/aced_coarse_w_scale/coarse_anchors/field",  delete_black_uploads=True, cdn_cache=False, info=field_aced_cv_info)
#field_aced_cv.commit_info()
#field_aced_cv.add_scale([2,2,1], chunk_size=[128,128,1])
#field_aced_cv.commit_info()
field_aced_cv = CloudVolume("gs://zetta-research-nico/hippocampus/aced_coarse_w_scale/coarse_anchors/field", mip=[12288,12288,45], delete_black_uploads=True, cdn_cache=False)

for src_z, dst_z in ANCHOR_Z_MAP.items():
    field_aced_cv[:,:,dst_z,:] = field_aced_condensed_cv[:,:,src_z,:]

Uploading: 100%|██████████| 1/1 [00:00<00:00,  1.21it/s]


In [26]:
field_aced_cv[:,:,2236] = field_aced_cv[:,:,2740]

Uploading: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]
